In [13]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [14]:
!which miktex

/Users/yunlin/bin/miktex


In [15]:
!echo "\documentclass{article}\begin{document}Hello, MikTeX!\end{document}" > test.tex
!pdflatex test.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./test.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2022/texmf-dist/tex/latex/l3backend/l3backend-pdftex.def)
(./test.aux) [1{/usr/local/texlive/2022/texmf-var/fonts/map/pdftex/updmap/pdfte
x.map}] (./test.aux) )</usr/local/texlive/2022/texmf-dist/fonts/type1/public/am
sfonts/cm/cmr10.pfb>
Output written on test.pdf (1 page, 13411 bytes).
Transcript written on test.log.


In [16]:
!kpsewhich physics.sty

/usr/local/texlive/2022/texmf-dist/tex/latex/physics/physics.sty


$\usepackage{physics}$


$\pdv{f}{x}$

\vqty{a}

https://colab.research.google.com/drive/1nEjlDpHc1Seb7U2tu4A2Y-5w0GB6DTht?usp=sharing

Notes are written based on [Berkeley CS 285](https://rail.eecs.berkeley.edu/deeprlcourse/static/slides/lec-18.pdf) and [Rezende 2015 Variational Inference with Normalizing Flows](https://arxiv.org/abs/1505.05770)

<img src="presentation_img/abstract.png" width="400">

## Variational inference

#### Latent variable model

* Probabilistic models, $p(x)$, $p(y|x)$.
* Latent variables are neither evidence nor query.
* A classical model is mixture Gaussian, where the latent variable is $z = 1, 2, 3$, and $p(x) = \underset{z}{\sum} p(x|z)p(z)$.
<img src="presentation_img/CS285_lec18_part1_0.png" width="400">

* How to train latent variable model? We want a maximum likelihood fit

\begin{equation}
    \theta \leftarrow \underset{\theta}{\text{argmax}} \frac{1}{N}\sum_i\log{p_\theta(x_i)},
\end{equation}
where
\begin{equation}
    p(x) = \int p(x|z)p(z)\,dz.
\end{equation}

\begin{gather}
    \Rightarrow \theta \leftarrow \underset{\theta}{\text{argmax}} \frac{1}{N}\sum_i\log{\left(\int p_\theta(x|z)p(z)\,dz\right)} 
    \label{eq:train-latent-var}
\end{gather}

* But the Equation \eqref{eq:train-latent-var} completely intractable especially in continuous models. So instead, we try to estimate the expected log-likelihood,

\begin{equation}
    \theta \leftarrow \underset{\theta}{\text{argmax}} \frac{1}{N}\sum_i E_{z\sim p(z|x_i)}[\log{p_\theta (x_i, z)}]
\end{equation}

#### The variational approximation

* How do we calculate $p(z|x_i)$? What if we can approximate with $q_i(z) = \mathcal{N}(\mu_i, \sigma_i)$, where $q_i$ is some simple distribution.
* With a $q_i$, you can construct a lower bound on $\log{p(x_i)}$. Then, maximizing this bound will push up $\log{p(x_i)}$.

\begin{align}
\log{p(x_i)} &= \log{\int_z p(x_i|z)p(z)} \nonumber \\
    &= \log{\int_z p(x_i|z)p(z)\frac{q_i(z)}{q_i(z)}} \nonumber \\
    &= \log{E_{z\sim q_i(z)}\left[\frac{p(x_i|z)p(z)}{q_i(z)}\right]} \nonumber \\
    &\ge E_{z\sim q_i(z)}\left[\log{\frac{p(x_i|z)p(z)}{q_i(z)}}\right] \quad \text{use Jensen's inequality for concave functions (log)} \log{E[y]} \ge E[\log{y}] \nonumber\\
    &= E_{z\sim q_i(z)}[\log{p(x_i|z)} + \log{p(z)}] - E_{z\sim q_i(z)}[\log{q_i(z)}] \quad \text{equation stands for all $q_i$} \nonumber \\
    &= E_{z\sim q_i(z)}[\log{p(x_i|z)} + \log{p(z)}] - \mathcal{H}(q_i(z)) \quad \text{last term is "entropy"} \nonumber \\
    &= E_{z\sim q_i(z)}[\log{p(x_i|z)}] - D_{\rm KL}(q_i(z)||p(z)) \label{eq:ELBO1} \quad \text{last term is KL divergence} \\
    &= \mathcal{L}_i \label{eq:Li},
\end{align}where $\mathcal{L}_i$ is called the evidence lower bound (ELBO).

#### A brief aside of entropy

* $\mathcal{H}$ is the entropy, 
\begin{equation}
    \mathcal{H(q)} = -E_{x\sim q(x)}[\log{q(x)}] = -\int_x q(x)\log{q(x)} \,dx
\end{equation}
    * Intuition 1: how random is the random variable?
    * Intuition 2: how large is the log probability in expectation under itself?
    
#### A brief aside of KL divergence

* A brief aside of KL divergence 
\begin{equation}
    D_{\rm KL}(q(z)||p(z)) = E_{z\sim q(z)}\left[\log{\frac{q(z)}{p(z)}}\right] = E_{x\sim q(z)}[\log{q(z)}] - E_{z\sim q(z)}[\log{p(z)}] = -E_{z\sim q(z)}[\log{p(z)}] - \mathcal{H}(q)
\end{equation}
    * Intuition 1: how different are two distributions?
    * Intuition 2: how small is the expected log probability of one distribution under another, minus entropy?
* KL divergence always $\ge 0$.
    * Derivation ...


#### The variational approximation (cont.)

* Equations \eqref{eq:ELBO1} and \eqref{eq:Li} suggest that we maximize $\mathcal{L}_i(p, q_i)$.
* On the other hand,
\begin{align}
    &... \text{leave out derivation (slides 15, 16)} \nonumber \\
    \Rightarrow \quad & D_{\rm KL}(q_i(z)||p(z|x_i)) = -\mathcal{L}_i (p, q_i) + \log{p(x_i)} \label{eq:algorithm}
\end{align}
* Note the $D_{\rm KL}$ terms of Equations \eqref{eq:ELBO1} and \eqref{eq:algorithm} are different, but the $\mathcal{L}_i$ terms are the same.
* <font color=blue>The point is Equation \eqref{eq:algorithm} immediately suggests a practical learning algorithm: Take the variational lower bound, maximize w.r.t. $q_i$ to get the tightest bound (also minimize KL divergence), and then maximize w.r.t. $p$ to improve the likelihood. Then alternate these two steps.</font>
    * Our goal was $\theta \leftarrow \underset{\theta}{\text{argmax}} \frac{1}{N}\sum_i \log{p_\theta (x_i)}$ but it is intractable, so instead we do $\theta \leftarrow \underset{\theta}{\text{argmax}} \frac{1}{N}\sum_i \log{\mathcal{L}_i(p, q_i)}$
    * For each $x_i$ (or mini-batch):
        * Calculate $\nabla_\theta \mathcal{L}_i (p, q_i)$
            * Sample $z \sim q_i(z)$ 
            * $\nabla_\theta \mathcal{L}_i (p, q_i) \approx \nabla_\theta\log{p_\theta}(x_i|z) \quad$ Use the sample to estimate the gradient.
        * Improve $\theta$, $\theta \leftarrow \theta + \alpha \nabla_\theta\mathcal{L}_i(p, q_i)$
        * Update $q_i$ to maximize the same evidence lower bound $\mathcal{L}_i (p, q_i) \leftarrow$ How to do this? 
        
* In the last step, let's say $q_i(z) = \mathcal{N}(\mu_i, \sigma_i)$, and it's possible to do $\nabla_{\mu_i} \mathcal{L}$ and $\nabla_{\sigma_i} \mathcal{L}$ to optimize $\mu_i, \sigma_i$. But it's not scalable for large dataset. There are in total $|\theta| + (|\mu_i| + |\sigma_i|)\times N$ parameters.



* To execute the algorithm above effectively, we need to solve the following two problems:
    * Efficient computation of the derivative of the first term of Equation \eqref{eq:ELBO1}: $\nabla_\phi E_{z\sim q_\phi(z|x_i)}[\log{p_\theta(x_i|z)}]$.
        * What about the KL divergence term? <font color=red>If you assume $q_i(z)$ and $p(z)$ both to be Gaussian, then the KL divergence is analytically known.</font>
    * Choosing the richest, computationally-feasible approximation of posterior distribution $q$. 
* Solutions:
    * Problem 1: Amortized variational inference $\rightarrow$ Monte Carlo gradient estimate + inference networks.
    * Problem 2: Normalizing flow

## Amortized variational inference

* Instead of having a separate $q_i(z)$ for every data point $x_i$, we have one network $q(z|x_i)$, which aims to approximate posterior.
* Then in our generative model, we have one network that maps from $z$ to $x$ ($p_\theta(x|z)$), and another network that maps from $x$ to $z$ ($q_\phi(z|x) = \mathcal{N}(\mu_\phi(x), \sigma_\phi(x))$). That's the idea behind amortized variational inference.

<img src="presentation_img/CS285_lec18_part3_network.png" width="700">

* The ELBO becomes 
\begin{equation}
    \mathcal{L}_i = \mathcal{L}(p_\theta(x_i|z), q_\phi(z|x_i)) = E_{\color{brown}{z\sim q_\phi(z|x_i)}}[\log{p_\theta(x_i|z)}] - D_{\rm KL}(\color{brown}{q_\phi(z|x_i)}||p(z)) \label{eq:ELBO2}. 
\end{equation}
* The algorithm above becomes 
    * For each $x_i$ (or mini-batch):
        * Calculate $\nabla_\theta \mathcal{L}(p_\theta(x_i|z), q_\phi(z|x_i))$
            * Sample $z \sim \color{brown}{q_\phi(z|x_i)}$
            * $\nabla_\theta \mathcal{L}_i (p, q_i) \approx \nabla_\theta\log{p_\theta}(x_i|z)$
        * $\theta \leftarrow \theta + \alpha \nabla_\theta\mathcal{L}_i(p, q_i)$
        * $\color{brown}{\phi \leftarrow \phi + \alpha \nabla_\phi\mathcal{L}_i(p, q_i)}$




#### Reparameterization trick

* Parameterize $z = \mu_i + \epsilon \sigma_i$, and draw $\epsilon\sim \mathcal{N}(0, 1)$.
* <font color=red>In reinforcement learning, we cannot calculate the derivatives through dynamics. But in amortized VI, there's no unknown dynamics. There's only $q$. And calculate derivatives through mean and variance of $q$ is feasible.</font>
* Express $z$ as a deterministic function, parameterized by $\phi$, of another random variable $\epsilon$ that is independent of $\phi$. <font color=red>When we do this, we can get a better gradient estimate. ...... $\phi$ here only parameterizes a deterministic quantity. We can write a better gradient estimator based on this expectation of $\epsilon$.</font>
* The ELBO
\begin{align}
    \mathcal{L}_i &= E_{z\sim q_\phi(z|x_i)}[\log{p_\theta(x_i|z)}] - D_{\rm KL}(q_\phi(z|x_i)||p(z)) \label{eq:ELBO3} \\
                  &= E_{z\sim \mathcal{N}(0, 1)}[\log{p_\theta(x_i|\mu_\phi(x_i)) + \epsilon\sigma_\phi(x_i)}] - D_{\rm KL}(q_\phi(z|x_i)||p(z)) \\
                  &\approx \log{p_\theta}(x_i|\mu_\phi + \epsilon\sigma_\phi(x_i)) - D_{\rm KL}(q_\phi(z|x_i)||p(z)) .
\end{align}

<img src="presentation_img/CS285_lec18_part3_repara_network.png" width="700">

* Why the ELBO (Equation \eqref{eq:ELBO3}) works well?
    * Reconstruction term and KL divergence.

## Normalizing flow

* In Equation \eqref{eq:ELBO3}, the optimal variational distribution allows $D_{\rm KL}(q||p) = 0$, which means $q_\phi(z|x) = p_\theta(z|x)$. 
* But this is impossible as generally we assign $q$ a Gaussian or other mean-field approximation.
* Normalizing flow allows one to obtain variational distributions that are highly flexible and preferably close to the true posterior distribution, by transforming a probability density through a sequence of invertible mappings.

\begin{equation}
    q(z') = q(z) \left|\det{\frac{\partial f^{-1}}{\partial{z}'}}\right| = q(z)\left|\det{\frac{\partial f}{\partial{z}}}\right| ^{-1} \label{eq:transform-q}
\end{equation}

* We can achieve arbitrarily complex densities by successively applying simple maps through Equation \eqref{eq:transform-q}.
* After $K$ times of transformations

\begin{equation}
    \ln{q_K(\mathbf{z}_K)} = \ln{q_0(\mathbf{z}_0)} - \sum_{k=1}^K \ln{\left|\frac{\partial f_k}{\partial{\mathbf{z}_{k-1}}}\right|},
\end{equation}

where
\begin{equation}
    \mathbf{z}_K = f_K \circ ... \circ f_2 \circ f_1(\mathbf{z}_0)
\end{equation}
* The initial distribuiton $q_0(\mathbf{z}_0)$ is called the flow, and the path formed by distributions $q_k$ is a normalizing flow.

Example: [Chatterjee 2022 Pre-merger sky localization of gravitational waves from binary neutron star mergers using deep learning](https://arxiv.org/abs/2301.03558)

<img src="presentation_img/model_architect.png" width="900">